# Chapter 9: 2D axissymmetric tri element
Daryl Logan: A first course in the finite element method, 6th edition, 2020, Chapter 9.

Note: This is not an isoparametric formulation and the way the stiffenss matrix is calculated at the end is also not the best.

In [1]:
import numpy as np
import sympy as sp
sp.init_printing()

### The 2D axissymmetric tri element

This is a triangular element with nodes: i, j, m.

Coordinates are: r, z for radial and vertical directions, respectively.

Each node hat 2 degrees of freedom (DOFs):
- u: radial displacement
- w: vertical displacement

The DOFs are ordered as follows: u, w.

### The physical properties of the element

In [2]:
E = 2.0e5  # Young's modulus in Pa
nu = 0.25  # Poisson's ratio
t = 1000  # thickness in mm
t /= 1000  # convert to m

# Define symbolic physical coordinates
r1, z1, r2, z2, r3, z3 = sp.symbols('r1 z1 r2 z2 r3 z3')

# Example coordinates for the nodes
# physical_coords = {'P1': (0, 0),
#                    'P2': (1, 0),
#                    'P3': (0, 1)}  # example coordinates for the nodes
physical_coords = {'P1': (4, 0),
                   'P2': (6, 2),
                   'P3': (3, 1)}  # example coordinates for the nodes
physical_coords = {'P1': (0, 0),
                   'P2': (3, 0),
                   'P3': (0, 2)}  # example coordinates for the nodes

physical_coords_map = {'r1': physical_coords['P1'][0], 'z1': physical_coords['P1'][1],
                                 'r2': physical_coords['P2'][0], 'z2': physical_coords['P2'][1],
                                  'r3': physical_coords['P3'][0], 'z3': physical_coords['P3'][1]}

R, Z = sp.symbols('R Z')  # radial and vertical coordinates

# Define the physical nodes as a matrix so it can be used a symbolic evaluation
physical_nodes = sp.Matrix([[r1, z1], 
                            [r2, z2], 
                            [r3, z3]])

Rv = sp.Matrix([r1, r2, r3])  # radial coordinate vector
Zv = sp.Matrix([z1, z2, z3])  # vertical coordinate vector


In [3]:
ND = 2  # number of degrees of freedom per node
NNODE = 3  # number of nodes

### Shape functions and derivatives

In [4]:
%matplotlib inline

def alpha():
    """Shape function for the axissymmetric tri element."""
    return sp.Matrix([
        Rv[1] * Zv[2] - Rv[2] * Zv[1],
        Rv[2] * Zv[0] - Rv[0] * Zv[2],
        Rv[0] * Zv[1] - Rv[1] * Zv[0],
    ])

def beta():
    """Shape function for the axissymmetric tri element."""
    return sp.Matrix([
        Zv[1] - Zv[2],
        Zv[2] - Zv[0],
        Zv[0] - Zv[1],
    ])

def gamma():
    """Shape function for the axissymmetric tri element."""
    return sp.Matrix([
        Rv[2] - Rv[1],
        Rv[0] - Rv[2],
        Rv[1] - Rv[0],
    ])

def shape_functions():
    _alpha = alpha()
    _beta = beta()
    _gamma = gamma()

    two_A = Rv.dot(_beta) # same as r1*beta1 + r2*beta2 + r3*beta3

    return sp.Matrix([
        (_alpha[0] + _beta[0] * R + _gamma[0] * Z) / two_A,
        (_alpha[1] + _beta[1] * R + _gamma[1] * Z) / two_A,
        (_alpha[2] + _beta[2] * R + _gamma[2] * Z) / two_A,
        ])


# Calculate the derivatives by s and t, symbolically
def shape_function_derivatives() -> np.array:
    """
    Shape function derivatives.
    [0] = dNi/ds, [1] = dNi/dt
    """
    sh = shape_functions()  # N1, N2, N3
    dN_dr = []
    dN_dz = []

    for i in range(len(sh)):
        dN_dr.append(sp.diff(sh[i], R))
        dN_dz.append(sp.diff(sh[i], Z))

    return dN_dr, dN_dz

# The symbolic shape functions
N_vec = shape_functions()

# 2. Substituting the symbolic coordinates with the physical coordinates
# This will give us the shape functions for the specific element defined by the physical coordinates
N_specific_element = N_vec.subs(physical_coords_map)

print("Symbolic shape function for this element:")
sp.pprint(N_specific_element)

# At node 1 N = [1, 0, 0]
print("\nValues at Node 1:")
sp.pprint(N_specific_element.subs({'R': physical_coords['P1'][0], 'Z': physical_coords['P1'][1]}).T)

# At node 2 N = [0, 1, 0]
print("\nValues at Node 2:")
sp.pprint(N_specific_element.subs({'R': physical_coords['P2'][0], 'Z': physical_coords['P2'][1]}).T)

# At node 3 N = [0, 0, 1]
print("\nValues at Node 3:")
sp.pprint(N_specific_element.subs({'R': physical_coords['P3'][0], 'Z': physical_coords['P3'][1]}).T)

print('\nShape function derivatives:')
dN_dr, dN_dz = shape_function_derivatives()
for i in range(len(dN_dr)):
    print(f'dN{i+1}/dr = {dN_dr[i]}')
    print(f'dN{i+1}/dz = {dN_dz[i]}')

# the shape function derivatives for the specific element
dN_dr_e = dN_dr[0].subs(physical_coords_map)
dN_dz_e = dN_dz[0].subs(physical_coords_map)
sp.pprint(dN_dr_e)
sp.pprint(dN_dz_e)

Symbolic shape function for this element:
⎡  R   Z    ⎤
⎢- ─ - ─ + 1⎥
⎢  3   2    ⎥
⎢           ⎥
⎢     R     ⎥
⎢     ─     ⎥
⎢     3     ⎥
⎢           ⎥
⎢     Z     ⎥
⎢     ─     ⎥
⎣     2     ⎦

Values at Node 1:
[1  0  0]

Values at Node 2:
[0  1  0]

Values at Node 3:
[0  0  1]

Shape function derivatives:
dN1/dr = (z2 - z3)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
dN1/dz = (-r2 + r3)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
dN2/dr = (-z1 + z3)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
dN2/dz = (r1 - r3)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
dN3/dr = (z1 - z2)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
dN3/dz = (-r1 + r2)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
-1/3
-1/2


### The shape function matrix

The shape function matrix $N$ is a matrix that contains the shape functions for each node. It has the structure:
$$
N = \begin{bmatrix}
N_i & 0 & N_j & 0 & N_m & 0 \\
0 & N_i & 0 & N_j & 0 & N_m\\
\end{bmatrix}
$$

In [5]:
def N() -> np.array:
    """
    shape function matrix N.
    It has the structure:
    [[Ni, 0, Nj, 0, Nm, 0],
     [0, Ni, 0, Nj, 0, Nm]]

    """
    _N = shape_functions()  # the shape functions

    # preparing the return matrix
    _ret = sp.Matrix(ND, ND * NNODE, lambda i, j: 0)  # ND rows, ND * NNODE columns

    # filling the matrix
    for i in range(NNODE):
        for j in range(ND):
            _ret[j, j + ND * i] = _N[i]

    return _ret

_N = N()
print("The shape function matrix N is {}:".format(_N.shape))
print()
print('The shape function matrix N for this element:')
_N_e = _N.subs(physical_coords_map)
sp.pprint(_N_e)

print('\nThe shape function matrix N for this element at node 1:')
midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0]) / 3, \
            (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1]) / 3
_N_e_node1 = _N_e.subs({'R': midpoint[0], 'Z': midpoint[1]})
sp.pprint(_N_e_node1.applyfunc(lambda x: x.round(4)))


The shape function matrix N is (2, 6):

The shape function matrix N for this element:
⎡  R   Z                   R     Z   ⎤
⎢- ─ - ─ + 1       0       ─  0  ─  0⎥
⎢  3   2                   3     2   ⎥
⎢                                    ⎥
⎢               R   Z         R     Z⎥
⎢     0       - ─ - ─ + 1  0  ─  0  ─⎥
⎣               3   2         3     2⎦

The shape function matrix N for this element at node 1:
⎡0.3333    0     0.3333    0     0.3333    0   ⎤
⎢                                              ⎥
⎣  0     0.3333    0     0.3333    0     0.3333⎦


### $\begin{bmatrix}B\end{bmatrix}$ matrix
The full B matrix

In [6]:
def B_symbolic():

    _alpha = alpha()
    _beta = beta()
    _gamma = gamma()

    two_A = Rv.dot(_beta) # same as r1*beta1 + r2*beta2 + r3*beta3

    B = sp.zeros(4, ND*NNODE)
    for i in range(NNODE):
        B[0, 2*i] = _beta[i] / two_A
        B[1, 2*i+1] = _gamma[i] / two_A
        B[2, 2*i] = (_alpha[i] / R + _beta[i] + _gamma[i] * Z / R) / two_A
        B[3, 2*i] = _gamma[i] / two_A
        B[3, 2*i+1] = _beta[i] / two_A

    return B

# The symbolic B matrix
B = B_symbolic()

# the B matrix for this element
B_e = B.subs(physical_coords_map)
# the B matrix for this element at the midpoint
print('the B matrix for this element at the midpoint')
sp.pprint(B_e.subs({'R': midpoint[0], 'Z': midpoint[1]}))

# the B matrix for this element at node 1. Note, the node 1 is at the origin (0, 0) so we get multiple nan values due to division by zero
print('the B matrix for this element at node 1. Note, the node 1 is at the origin (0, 0) so we get multiple nan values due to division by zero')
sp.pprint(B_e.subs({'R': physical_coords['P1'][0], 'Z': physical_coords['P1'][1]}))

the B matrix for this element at the midpoint
⎡      -1/3          0    1/3   0           0           0 ⎤
⎢                                                         ⎥
⎢        0          -1/2   0    0           0          1/2⎥
⎢                                                         ⎥
⎢0.333333333333333   0    1/3   0   0.333333333333333   0 ⎥
⎢                                                         ⎥
⎣      -1/2         -1/3   0   1/3         1/2          0 ⎦
the B matrix for this element at node 1. Note, the node 1 is at the origin (0, 0) so we get multiple nan values due to division by zero
⎡-1/3   0    1/3   0    0    0 ⎤
⎢                              ⎥
⎢ 0    -1/2   0    0    0   1/2⎥
⎢                              ⎥
⎢nan    0    1/3   0   nan   0 ⎥
⎢                              ⎥
⎣-1/2  -1/3   0   1/3  1/2   0 ⎦


### The constitutive matrix $\begin{bmatrix}D\end{bmatrix}$

In [7]:
def D_matrix():
    """
    The constitutive matrix for the axissymmetric tri element.
    """
    # Plane stress condition
    D = sp.zeros(4, 4)
    D[0, 0] = 1 - nu
    D[0, 1] = nu
    D[0, 2] = nu
    D[1, 0] = nu
    D[1, 1] = 1 - nu
    D[1, 2] = nu
    D[2, 0] = nu
    D[2, 1] = nu
    D[2, 2] = 1 - nu
    D[3, 3] = (1 - 2 * nu) / 2

    return D

### The stiffness matrix $\begin{bmatrix}K\end{bmatrix}$

In [8]:
def K_matrix_3():
    """
    The stiffness matrix for the axissymmetric tri element.
    This is method 3 from the book.
    It evaluates the B matrix at the midpoint of the element and then calculates the stiffness matrix.
    """
    K = sp.zeros(ND * NNODE, ND * NNODE)  # the stiffness matrix

    mp = midpoint

    B_e = B_symbolic().subs(physical_coords_map)
    B = B_e.subs({'R': mp[0], 'Z': mp[1]})  # evaluate B at the midpoint

    D = D_matrix().subs({nu: nu})  # the constitutive matrix

    _A = 0.5 * Rv.dot(beta()).subs(physical_coords_map) # same as r1*beta1 + r2*beta2 + r3*beta3

    for i in range(NNODE):
        for j in range(NNODE):
            K += (B.T * D * B) * 2 * np.pi * mp[0] * _A

    return K

# The stiffness matrix for this element
K_e = K_matrix_3()
K_e_formatted = K_e.applyfunc(lambda x: x.round(4))
print("The stiffness matrix K for this element:")
sp.pprint(K_e_formatted)

The stiffness matrix K for this element:
⎡29.4524   7.0686     0.0     -7.0686  -1.1781     0.0   ⎤
⎢                                                        ⎥
⎢7.0686    36.521   -14.1372  -4.7124  -14.1372  -31.8086⎥
⎢                                                        ⎥
⎢  0.0    -14.1372  37.6991      0     18.8496   14.1372 ⎥
⎢                                                        ⎥
⎢-7.0686  -4.7124      0      4.7124    7.0686      0    ⎥
⎢                                                        ⎥
⎢-1.1781  -14.1372  18.8496   7.0686    24.74     7.0686 ⎥
⎢                                                        ⎥
⎣  0.0    -31.8086  14.1372      0      7.0686   31.8086 ⎦


In [9]:
# the stiffness matrix as a numpy array
K_np = np.array(K_e).astype(np.float64)

# the eigenvalues
eigenvalues, eigenvectors = np.linalg.eigh(K_np)
print()
print('The eigenvalues of the stiffness matrix K are:')
eigenvalues = sorted([float(x) for x in eigenvalues])

print([round(ev, 5) for ev in eigenvalues])
# print(eigenvectors.shape)
# sp.pprint(eigenvectors)

# the eigenvectors for the near-zero eigenvalues
nearzero_eigenvalues = [ev for ev in eigenvalues if abs(ev) < 1e-10]
# printing the eigenvectors for the near-zero eigenvalues
print('The eigenvectors for the near-zero eigenvalues are:')
for i, ev in enumerate(nearzero_eigenvalues):
    print(f'Eigenvalue {i + 1}: {ev:.6f}')
    print('Eigenvector:')
    print(eigenvectors[:, i])
    print()


The eigenvalues of the stiffness matrix K are:
[-0.0, -0.0, 14.46519, 30.63053, 34.22128, 85.61662]
The eigenvectors for the near-zero eigenvalues are:
Eigenvalue 1: -0.000000
Eigenvector:
[-0.19971019  0.57175325 -0.19971019 -0.32694263  0.39942039  0.57175325]

Eigenvalue 2: -0.000000
Eigenvector:
[-0.1067609  -0.34902174 -0.1067609  -0.82944579  0.2135218  -0.34902174]

